In [1]:
%reload_ext autoreload
%autoreload 2
%reload_ext notexbook
%texify

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import time
import sys
import pathlib
import pickle
from pathlib import Path
from skimage.io import imread
import seaborn as sns
sns.set_style('white')
sys.setrecursionlimit(10000)
%matplotlib qt5

In [3]:
import cellbgnet
import cellbgnet.utils
from cellbgnet.datasets import DataSimulator
from cellbgnet.utils.hardware import cpu, gpu
from cellbgnet.model import CellBGModel
from cellbgnet.utils.plot_funcs import plot_od, plot_train_record

In [4]:
model_path = Path('/mnt/sda1/SMLAT/training_runs/good_models/model_8865_venus_low_ph.pkl')

In [5]:
with open(model_path, 'rb') as f:
    chromo_model = pickle.load(f)

In [6]:
chromo_model.evaluation_params['eval_imgs'].shape

(30, 1041, 1302)

In [7]:
plot_train_record(chromo_model)

#### Photon count range, get background average from simulation using model paramteres

In [8]:
mol_photons = (chromo_model.data_generator.simulation_params['min_photon'] + 1) /2 * chromo_model.data_generator.psf_params['photon_scale']

In [9]:
mol_photons

1875.0

#### Plot network outputs for the eval images after tiling and re-tiling of one eval image

In [11]:
from cellbgnet.analyze_eval import recognition, plot_full_img_predictions, assemble_full_img_predictions

In [12]:
eval_img = chromo_model.evaluation_params['eval_imgs'][10][np.newaxis,:]

In [13]:
plt.figure()
plt.imshow(eval_img[0], cmap='gray')
plt.colorbar()
plt.show()

In [14]:
fov_size = [eval_img.shape[2] * 65, eval_img.shape[1] * 65]

In [15]:
fov_size

[84630, 67665]

In [16]:
preds_tmp, n_per_img, plot_data = recognition(model=chromo_model, eval_imgs_all=eval_img,
                                             batch_size=16, use_tqdm=False,
                                             nms=True, candidate_threshold=0.3,
                                             nms_threshold=0.7, 
                                             pixel_nm=chromo_model.data_generator.psf_params['pixel_size_xy'],
                                             plot_num=1,
                                             win_size=128,
                                             padding=True,
                                             start_field_pos=[0, 0],
                                             padded_background=chromo_model.evaluation_params['padded_background'])

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]


In [17]:
plot_full_img_predictions(chromo_model, plot_infs=plot_data, eval_csv=None, plot_num=1, fov_size=fov_size, pixel_size=[65, 65])

#### Testing real image

In [18]:
images_dir = Path('/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_8865_2/venus/')
img_filenames = sorted(list(images_dir.glob('*.tiff')))

In [23]:
img_path = Path('/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_8865_2/venus/Pos32_img_000000125.tiff')

In [24]:
real_img = imread(img_path)[np.newaxis, :]


In [25]:
preds_tmp, n_per_img, plot_data = recognition(model=chromo_model, eval_imgs_all=real_img,
                                             batch_size=16, use_tqdm=False,
                                             nms=True, candidate_threshold=0.3,
                                             nms_threshold=0.7, 
                                             pixel_nm=chromo_model.data_generator.psf_params['pixel_size_xy'],
                                             plot_num=1,
                                             win_size=128,
                                             padding=True,
                                             start_field_pos=[0, 0],
                                             padded_background=chromo_model.evaluation_params['padded_background'])

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]


In [22]:
plot_full_img_predictions(chromo_model, plot_infs=plot_data, eval_csv=None, plot_num=1,
                        fov_size=fov_size, pixel_size=[65, 65])

In [21]:
len(preds_tmp)

366

#### Predictions are a list of numbers where each element corresponds to a localizaiton in the following order


    1. counter of the molecule per tile.. not a global counter on the frame, need to fix this
    2. image number used to index into the number of the image in the prediction arrays.
    Eval image is of shape [1, 1041, 1302], so, the image number will always be 1, if you give one image at a time
    3. x position in nm where 0 is top left corner
    4. y position in nm where 0 is top left corner
    5. z position in nm where 0 is from the reference 0 nm in height
    6. photon counts 
    7. probability afer nms
    8. x_sigma in nm 
    9. y_sigma in nm
    10. z_sigma in nm
    11. photon_counts_sigma
    12. x offset
    13. y offset


In [26]:
preds_tmp

[[1.0,
  1.0,
  25325.2421875,
  6982.552734375,
  37.81665802001953,
  1149.551513671875,
  1.011259913444519,
  14.265509605407715,
  15.97189712524414,
  47.59745788574219,
  168.12179565429688,
  0.11910994350910187,
  -0.07611070573329926],
 [2.0,
  1.0,
  26423.892578125,
  6981.2666015625,
  321.6169128417969,
  2036.275634765625,
  0.8009865880012512,
  18.079500198364258,
  19.683704376220703,
  51.808048248291016,
  354.2107238769531,
  0.021433474496006966,
  -0.09589913487434387],
 [1.0,
  1.0,
  57421.9921875,
  6918.7314453125,
  276.0229187011719,
  2122.123291015625,
  1.0016037225723267,
  12.278976440429688,
  17.620752334594727,
  37.742427825927734,
  277.9707946777344,
  -0.0847318097949028,
  -0.05797664821147919],
 [1.0,
  1.0,
  62967.7421875,
  6721.45947265625,
  228.1245574951172,
  1234.75244140625,
  0.7300586104393005,
  11.98918628692627,
  19.019424438476562,
  56.09822082519531,
  232.88429260253906,
  0.23447947204113007,
  -0.09293288737535477],
 [1.0

In [27]:
img_infs = assemble_full_img_predictions(chromo_model, plot_data)

In [28]:
img_infs.keys()

dict_keys(['Probs', 'XO', 'YO', 'ZO', 'Int', 'BG', 'XO_sig', 'YO_sig', 'ZO_sig', 'Int_sig', 'Probs_ps', 'XO_ps', 'YO_ps', 'ZO_ps', 'Samples_ps', 'raw_img', 'only_bg'])

In [25]:
for key, value in img_infs.items():
    plt.figure()
    plt.imshow(value)
    plt.colorbar()
    plt.title(key)
    plt.show()

In [30]:
plt.figure()
plt.imshow(img_infs['BG'], vmin=0, vmax=300, cmap='gray')
plt.colorbar()
plt.title('Predicted PSF')
plt.show()

In [25]:
plt.figure()
plt.imshow(img_infs['raw_img'], vmin=25, vmax=520 , cmap='gray')
plt.colorbar()
plt.title('Raw image')
plt.show()

In [51]:
plt.figure()
plt.imshow(img_infs['only_bg'][:-200, :-200])
plt.colorbar()
plt.title('Only bg')
plt.show()

##### Collect predictions of all emitters in one image for plotting distributions

In [65]:
x_global = np.array([x[2] for x in preds_tmp])
y_global = np.array([x[3] for x in preds_tmp])
z_global = np.array([x[4] for x in preds_tmp])
ph_global = np.array([x[5] for x in preds_tmp])
prob_dot_nms = np.array([x[6] for x in preds_tmp])
sigma_x = np.array([x[7] for x in preds_tmp])
sigma_y = np.array([x[8] for x in preds_tmp])
sigma_z = np.array([x[9] for x in preds_tmp])
sigma_ph = np.array([x[10] for x in preds_tmp])

#### Only plot z, photon counts and probablitiy so that you can check what is going on as time goes by

In [66]:
fig, ax = plt.subplots(nrows=1, ncols=2)

ax[0].hist(z_global, bins=50)
ax[0].axvspan(z_global.min(), z_global.max(), color='green', alpha=0.1)
ax[0].set_xlabel('z [nm] of the predicted emitters')
ax[0].set_ylabel('counts')

ax[1].hist(ph_global, bins=50)
ax[1].axvspan(ph_global.min(), ph_global.max(), color='green', alpha=0.1)
ax[1].set_xlabel('photon counts of the predicted emitters')
ax[1].set_ylabel('counts')

plt.suptitle(img_path)
plt.show()

### Plotting sigma-xyzi distributions of a single image (sigmas) predicted by the network

In [30]:

fig, ax = plt.subplots(nrows=2, ncols=2)
ax[0, 0].hist(sigma_x, bins=25)
ax[0, 0].axvspan(sigma_x.min(), sigma_x.max(), color='green', alpha=0.1)
ax[0, 0].set_xlabel(r'$\sigma_x$ [nm]')
ax[0, 0].set_ylabel('counts')
ax[0, 0].set_title(f"Mean sigma_x is {np.mean(sigma_x):0.3f}, min:{np.min(sigma_x):0.3f}, max:{np.max(sigma_x):0.3f}")

ax[0, 1].hist(sigma_y, bins=25)
ax[0, 1].axvspan(sigma_y.min(), sigma_y.max(), color='green', alpha=0.1)
ax[0, 1].set_xlabel(r'$\sigma_y$ [nm]')
ax[0, 1].set_ylabel('counts')
ax[0, 1].set_title(f"Mean sigma_y is {np.mean(sigma_y):0.3f}, min:{np.min(sigma_y):0.3f}, max:{np.max(sigma_y):0.3f}")


ax[1, 0].hist(sigma_z, bins=25)
ax[1, 0].axvspan(sigma_z.min(), sigma_z.max(), color='green', alpha=0.1)
ax[1, 0].set_xlabel(r'$\sigma_z$ [nm]')
ax[1, 0].set_ylabel('counts')
ax[1, 0].set_title(f"Mean sigma_z is {np.mean(sigma_z):0.3f}, min:{np.min(sigma_z):0.3f}, max:{np.max(sigma_z):0.3f}")

ax[1, 1].hist(sigma_ph, bins=25)
ax[1, 1].axvspan(sigma_ph.min(), sigma_ph.max(), color='green', alpha=0.1)
ax[1, 1].set_xlabel(r'$\sigma_{ph}$ [nm]')
ax[1, 1].set_ylabel('counts')
ax[1, 1].set_title(f"Mean sigma_ph is {np.mean(sigma_ph):0.3f}, min:{np.min(sigma_ph):0.3f}, max:{np.max(sigma_ph):0.3f}")

plt.show()

#### Plot distributions on predictions in one image xyzi

##### Loop over all images and plot variability in time

In [22]:
number_preds = []
for img_path in img_filenames:
    real_img = imread(img_path)[np.newaxis, :]
    preds_tmp, n_per_img, plot_data = recognition(model=chromo_model, eval_imgs_all=real_img,
                                             batch_size=16, use_tqdm=False,
                                             nms=True, candidate_threshold=0.1,
                                             nms_threshold=0.7, 
                                             pixel_nm=chromo_model.data_generator.psf_params['pixel_size_xy'],
                                             plot_num=1,
                                             win_size=128,
                                             padding=True,
                                             start_field_pos=[0, 0],
                                             padded_background=chromo_model.evaluation_params['padded_background'])
    number_preds.append(len(preds_tmp))
    print(img_path)

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000000.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000001.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000002.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000003.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000038.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000039.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000040.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000041.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000076.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000077.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000078.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000079.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000114.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000115.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000116.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000117.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000152.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000153.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000154.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000155.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/

...

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000418.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000419.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000420.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000421.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000456.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000457.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000458.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000459.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000494.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000495.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000496.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000497.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000532.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000533.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000534.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000535.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000570.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000571.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000572.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/SMLAT/data/real_data/chromosome_dots/pooled_3024/venus/Pos32_img_000000573.tiff
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]
/mnt/sda2/

In [25]:
plt.figure()
plt.plot(np.arange(0, len(number_preds)), number_preds)
plt.xlabel('Frame number')
plt.ylabel('Number of dots predicted')
plt.title('Chromosome dots 8865 Pos01')
plt.show()